Inicialize uma sessão Spark, que é a maneira de interagir com os dados no Spark

In [36]:
import json
from pyspark.sql import SparkSession
spark = SparkSession.builder.master(
    "local[1]"
    ).appName(
        'test'
    ).config(
        "spark.jars", "mariadb-java-client-1.8.0.jar"
    ).getOrCreate()

with open('param.json', 'r') as file:
    data = json.load(file)

properties = {
    "numPartitions": 10,
    "properties":{
        "user": "root",
        "password": "secret",
        "driver": "org.mariadb.jdbc.Driver"
    }
    
}
properties.update(**data['parametros_carga'])
print(json.dumps(data))

{"arquivo_sql": ["count.sql"], "parametros_query": {"tabela": "tera", "cidade": "Genevse"}, "parametros_carga": {"table": "tera", "url": "jdbc:mysql://localhost:3306/teste"}}


In [37]:
df = spark.read.jdbc(**properties).cache()
df.createOrReplaceTempView(data['parametros_carga']['table'])
parametros_query = data['parametros_query']
for file in data['arquivo_sql']:
    with open(file, 'r') as file:
        query = file.read()
    query = query.format_map(parametros_query)
    df_stg = spark.sql(query)
df_stg.count()

0